In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import numpy as np

import warnings
warnings.filterwarnings("ignore")
from datetime import date, datetime, timedelta

In [3]:
File_name = "spares_orders_2022-03-31 16:31:01.583067_to_2022-08-31 16:31:01.583067.csv"
data = pd.read_csv(File_name)
data=data[data['stockone_order_id'].notnull()]
data['title']=data['product_title']
data=data[data['user_type']=='retailer']
data['created_at'] = data['created_at'].astype('datetime64[ns]')
data=data[data['created_at']>=datetime(2022,4,1)]
data.shape

(120593, 44)

In [6]:
data['created_at'].min(), data['created_at'].max()

(Timestamp('2022-04-01 00:00:00'), Timestamp('2022-08-31 00:00:00'))

In [7]:
data.sample(10)

,Unnamed: 0,stockone_order_id,product_id,product_title,sku_code,sku_brand,mrp,price_per_quantity,available_quantity,total_discount,...,user_type,Delivery_address,user_id,sales_order_id,coupon_code,coupon_msg,key,so_items_warehouse_id,so_items_status,title
92470,92470,GMDQN538101,F1USTY08BA,"MGP Air Filter, ELEMENT,AIR CLEANER 13780M86J00",13780M86J00,MGP,1100.0,256.0,4.0,76.0,...,retailer,"narauli (Varanasi road ) azamgarh , near Tani...",4.034200e+04,SO-22230713-22606,GET20OFF,Coupon Applied Successfully,GMDQN538101_F1USTY08BA,888888888,forward,"MGP Air Filter, ELEMENT,AIR CLEANER 13780M86J00"
53158,53158,GMDAR500784,DLD9GJEA05,"Honda Bumper Bracket, BEAM R FR BUMPER UPPER 7...",71140T9DK10,HONDA,397.0,376.0,1.0,21.0,...,retailer,"671121, Abu Thayir , Home, Kolikkara House",8.487935e+07,SO-22230603-30230,NaN,NaN,GMDAR500784_DLD9GJEA05,999999999,forward,"Honda Bumper Bracket, BEAM R FR BUMPER UPPER 7..."
21342,21342,GMDTA473603,BZGEJ58TP0,HELIX HX8 5W-30 API SN PLUS FULLY SYNTHETIC EN...,550051520,SHELL,729.0,583.0,1.0,146.0,...,retailer,"141006, Ludhiana, Duggri, Ludhiana, Punjab, In...",8.468515e+07,SO-22230429-28370,GO500,Coupon Applied Successfully,GMDTA473603_BZGEJ58TP0,999999999,forward,HELIX HX8 5W-30 API SN PLUS FULLY SYNTHETIC EN...
17519,17519,GMDNO470335,GMS23176898,"Zip Cabin Filter, CABIN AIR FILTER ZC6076",ZC6076,ZIP,200.0,140.0,1.0,60.0,...,retailer,"T D gutta , 1-8-211/a, Home, MAHABUBNAGAR, z...",8.487346e+07,SO-22230426-77958,NaN,NaN,GMDNO470335_GMS23176898,999999999,back,"Zip Cabin Filter, CABIN AIR FILTER ZC6076"
21664,21664,GMDOU473911,P506HVVZ04,"Subros Condenser, CONDENSER ASSEMBLY 64770044900H",64770044900H,SUBROS,8546.0,3418.0,2.0,1710.0,...,retailer,"Shankaramattam Road, Santhipuram, 49-51-6/1, ...",8.485926e+07,SO-22230429-63511,AUTOPART,Coupon Applied Successfully,GMDOU473911_P506HVVZ04,888888888,forward,"Subros Condenser, CONDENSER ASSEMBLY 64770044900H"
5105,5105,GMDCY459580,YCWW7TRCIQ,"VAG (Volkswagen, Skoda, Audi) Water Pump, WATE...",03L121011P,"VAG (VOLKSWAGEN, SKODA, AUDI)",2239.0,2239.0,1.0,0.0,...,retailer,"opp planery archade , near bombay hotel chowk ...",8.472480e+07,SO-22230408-40333,SPARES15,Coupon Applied Successfully,GMDCY459580_YCWW7TRCIQ,999999999,forward,"VAG (Volkswagen, Skoda, Audi) Water Pump, WATE..."
64473,64473,GMDWQ509841,R4DGRYEKNL,"MGP Clutch Friction Disc, DISC, CLUTCH(EECO) 2...",22400M83B20,MGP,1095.0,1095.0,1.0,0.0,...,retailer,"Chaitanyapure, sutahata, Pabitra Transport, op...",8.486987e+07,SO-22230614-99454,GET20OFF,Coupon Applied Successfully,GMDWQ509841_R4DGRYEKNL,888888888,forward,"MGP Clutch Friction Disc, DISC, CLUTCH(EECO) 2..."
68854,68854,GMDWI513247,6XAJJAN36H,"MGP Side View Mirror Cover, COVER, MIRROR VISO...",84728M74LA05PK,MGP,230.0,230.0,1.0,0.0,...,retailer,"Allahabad Pratapgarh Road, Ravi Telecom Patel ...",2.002663e+09,SO-22230618-98487,NaN,NaN,GMDWI513247_6XAJJAN36H,888888888,forward,"MGP Side View Mirror Cover, COVER, MIRROR VISO..."
91279,91279,GMDXX536846,GMS23175900,"Toyota Front Brake Pad, BRAKE PAD, MVP 04465YZZS7",04465YZZS7,TOYOTA,5878.0,2880.0,2.0,118.0,...,retailer,"C/18B, GROUND FLOOR,OPP. SWASTIK CHAMBERS, CST...",8.488237e+07,SO-22230712-37020,GET20OFF,Coupon Applied Successfully,GMDXX536846_GMS23175900,999999999,forward,"Toyota Front Brake Pad, BRAKE PAD, MVP 04465YZZS7"
124435,124435,GMDPV570387,220721054333817VZY,"MGP Speaker, SPEAKER ASSY 39102M55R01",39102M55R01,MGP,880.0,440.0,2.0,0.0,...,retailer,"Paratha Colony, 01, ANANTHPUR, ALLABAKASH ABS,...",1.202321e+09,SO-22230814-82231,NaN,NaN,GMDPV570387_220721054333817VZY,999999999,forward,"MGP Speaker, SPEAKER ASSY 39102M55R01"


In [4]:
data.columns

Index(['Unnamed: 0', 'stockone_order_id', 'product_id', 'product_title',
       'sku_code', 'sku_brand', 'mrp', 'price_per_quantity',
       'available_quantity', 'total_discount', 'cancelled_quantity',
       'total_coupon_discount', 'price', 'is_accessories', 'category_name',
       'segment_name', 'quantity', 'created_at', 'updated_at', 'order_status',
       'payment_mode', 'channel_reference', 'delivery_charges', 'gm_order_id',
       'coupon_discount', 'marked_ttp', 'order_price', 'order_tax_amount',
       'order_total', 'phone_number', 'first_name', 'pincode',
       'shipping_state', 'email', 'user_type', 'Delivery_address', 'user_id',
       'sales_order_id', 'coupon_code', 'coupon_msg', 'key',
       'so_items_warehouse_id', 'so_items_status', 'title'],
      dtype='object')

In [8]:
data[data['coupon_code']=='FREEDOT4']

,Unnamed: 0,stockone_order_id,product_id,product_title,sku_code,sku_brand,mrp,price_per_quantity,available_quantity,total_discount,...,user_type,Delivery_address,user_id,sales_order_id,coupon_code,coupon_msg,key,so_items_warehouse_id,so_items_status,title
189,189,GMDNK454708,GJFAMPJSFP,LIP SEAL,7701071156,"RNM ALLIANCE (RENAULT, NISSAN, MITSUBISHI)",619.0,588.0,1.0,31.0,...,retailer,"Rohini North West Delhi, 106 abhilasha apartme...",7.858961e+06,SO-22230408-42363,FREEDOT4,Coupon Applied Successfully,GMDNK454708_GJFAMPJSFP,999999999,back,LIP SEAL
190,190,GMDNK454708,GMS23177589,GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid ...,GMUNZZBO002,GOMECHANIC,119.0,0.0,1.0,119.0,...,retailer,"Rohini North West Delhi, 106 abhilasha apartme...",7.858961e+06,SO-22230408-42363,FREEDOT4,Coupon Applied Successfully,GMDNK454708_GMS23177589,999999999,forward,GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid ...
224,224,GMDWQ454770,GMS23177589,GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid ...,GMUNZZBO002,GOMECHANIC,119.0,0.0,1.0,119.0,...,retailer,"Serilingampalle (M), Ranga Reddy, Telangana, f...",2.920900e+04,SO-22230408-27964,FREEDOT4,Coupon Applied Successfully,GMDWQ454770_GMS23177589,666666666,forward,GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid ...
236,236,GMDMX454778,GMS23177589,GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid ...,GMUNZZBO002,GOMECHANIC,119.0,0.0,1.0,119.0,...,retailer,"Nagpur Road Chandangaon Chhindwara, pump house...",3.667626e+08,SO-22230408-20699,FREEDOT4,Coupon Applied Successfully,GMDMX454778_GMS23177589,999999999,forward,GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid ...
237,237,GMDMX454778,HJN90ZHA24,"Ford Timing Belt/Chain, BELT ALTERNATIVE PART ...",2S696K288AC,FORD,1342.0,1302.0,1.0,40.0,...,retailer,"Nagpur Road Chandangaon Chhindwara, pump house...",3.667626e+08,SO-22230408-20699,FREEDOT4,Coupon Applied Successfully,GMDMX454778_HJN90ZHA24,999999999,forward,"Ford Timing Belt/Chain, BELT ALTERNATIVE PART ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111532,111532,GMDIZ558068,TE9U7SZCL9,"MGP Spring, SPRINGRELEASE SHAFT RETURN 23269M7...",23269M76M00,MGP,95.0,95.0,0.0,0.0,...,retailer,"Bhamian Khurd Ludhiana, plot No 32 , G S colon...",1.435276e+09,SO-22230802-20261,FREEDOT4,Coupon Applied Successfully,GMDIZ558068_TE9U7SZCL9,999999999,back,"MGP Spring, SPRINGRELEASE SHAFT RETURN 23269M7..."
111533,111533,GMDIZ558068,U82QPE7JMF,"MGP Seal Injection Pump, GASKETFUEL PUMP 15111...",15111M50G10,MGP,57.0,57.0,0.0,0.0,...,retailer,"Bhamian Khurd Ludhiana, plot No 32 , G S colon...",1.435276e+09,SO-22230802-20261,FREEDOT4,Coupon Applied Successfully,GMDIZ558068_U82QPE7JMF,999999999,back,"MGP Seal Injection Pump, GASKETFUEL PUMP 15111..."
111534,111534,GMDIZ558068,WYKGP0B8A3,GoMechanic WINDSHIELD WASHER (50 ML) Windshiel...,GMUNZZWS001,GOMECHANIC,60.0,15.0,3.0,15.0,...,retailer,"Bhamian Khurd Ludhiana, plot No 32 , G S colon...",1.435276e+09,SO-22230802-20261,FREEDOT4,Coupon Applied Successfully,GMDIZ558068_WYKGP0B8A3,999999999,forward,GoMechanic WINDSHIELD WASHER (50 ML) Windshiel...
111559,111559,GMDKS558111,GMS22807333,"Exxonmobil Transmission Oil, MOBIL LUBE GX 80W...",139471,EXXONMOBIL,861.0,672.0,1.0,189.0,...,retailer,"At-Ramnagar colony,shekaiwadi, Post -Taluka-Ak...",1.446378e+09,SO-22230802-62225,FREEDOT4,Coupon Applied Successfully,GMDKS558111_GMS22807333,999999999,forward,"Exxonmobil Transmission Oil, MOBIL LUBE GX 80W..."


In [9]:
data['user_type'].unique()

array(['retailer'], dtype=object)

In [11]:
data.columns

Index(['Unnamed: 0', 'stockone_order_id', 'product_id', 'product_title',
       'sku_code', 'sku_brand', 'mrp', 'price_per_quantity',
       'available_quantity', 'total_discount', 'cancelled_quantity',
       'total_coupon_discount', 'price', 'is_accessories', 'category_name',
       'segment_name', 'quantity', 'created_at', 'updated_at', 'order_status',
       'payment_mode', 'channel_reference', 'delivery_charges', 'gm_order_id',
       'coupon_discount', 'marked_ttp', 'order_price', 'order_tax_amount',
       'order_total', 'phone_number', 'first_name', 'pincode',
       'shipping_state', 'email', 'user_type', 'Delivery_address', 'user_id',
       'sales_order_id', 'coupon_code', 'coupon_msg', 'key',
       'so_items_warehouse_id', 'so_items_status', 'title'],
      dtype='object')

In [12]:
data['coupon_code'].unique()

array([nan, 'FREEDOT4', 'SPARES15', 'FREEDOT3', 'GO500', 'LUMAX10',
       'AUTOGOLD5', 'BOSCH10', 'AUTOGOLD10', '1.5LCART', 'VALEO10',
       'PUROLATOR10', '2022', 'SRF10', 'TVS10', 'AUTO29', 'GOMECH10',
       'TOYOTA3', 'FREECABLE', 'LUMAX5', 'TOYOTA7', 'GOODFRIDAY',
       'SUPERFAST', 'AUTOPART', 'CART5', 'HOLYDAY', 'CARSAVER', 'CELEB6',
       'GET20OFF', 'SAVEBIG', 'MONSOON', 'GOSPARES', 'FREEDOM', 'IND75'],
      dtype=object)

In [13]:
data=data[~((data['coupon_code']=='FREEDOT3') & (data['sku_code']=='GMUNZZBO001'))]
data=data[~((data['coupon_code']=='FREEDOT4') & (data['sku_code']=='GMUNZZBO002'))]
data

,Unnamed: 0,stockone_order_id,product_id,product_title,sku_code,sku_brand,mrp,price_per_quantity,available_quantity,total_discount,...,user_type,Delivery_address,user_id,sales_order_id,coupon_code,coupon_msg,key,so_items_warehouse_id,so_items_status,title
179,179,GMDEF454696,GMS23176898,"Zip Cabin Filter, CABIN AIR FILTER ZC6076",ZC6076,ZIP,200.0,200.0,1.0,0.0,...,retailer,"Chenkottukonam, Rahul Nivas anandeswaram chenk...",1.789314e+09,SO-22230401-38945,NaN,NaN,GMDEF454696_GMS23176898,999999999,back,"Zip Cabin Filter, CABIN AIR FILTER ZC6076"
180,180,GMDOZ454697,RIPO2I3KCX,MIRROR-OTR RH,963019273R,"RNM ALLIANCE (RENAULT, NISSAN, MITSUBISHI)",1346.0,1279.0,0.0,67.0,...,retailer,"BALAJIPETA \nS.V.ROYAL PLAZA\nG2 , Door no 10...",3.747596e+07,SO-22230405-58494,NaN,NaN,GMDOZ454697_RIPO2I3KCX,888888888,back,MIRROR-OTR RH
181,181,GMDPV454698,TOD14XQHLS,"MGP Fuel Lid Cable, CABLE COMPFUEL LID OPENER ...",76430M53M00,MGP,130.0,130.0,1.0,0.0,...,retailer,"WW72+6QQ, Shinde, Nashik, Hotel Morya Family R...",1.840752e+09,SO-22230401-73844,NaN,NaN,GMDPV454698_TOD14XQHLS,999999999,back,"MGP Fuel Lid Cable, CABLE COMPFUEL LID OPENER ..."
182,182,GMDBA454703,IIT886E0JB,"Honda Grille, MOLDING FR GRILLE 71122TM0T11",71122TM0T11,HONDA,4737.0,4737.0,1.0,0.0,...,retailer,"Rangatand, Dhanbad, Dhanbad, shankar mall, DHA...",1.257051e+09,SO-22230401-45525,NaN,NaN,GMDBA454703_IIT886E0JB,999999999,forward,"Honda Grille, MOLDING FR GRILLE 71122TM0T11"
183,183,GMDEV454704,HOVRCEGW6S,"Autogold Fog Lamp, FOG LAMP ASSY VERNA FLUDIC ...",AGF0721L,AUTOGOLD,965.0,676.0,0.0,289.0,...,retailer,"71, chandan colony , Ranjhi jabalpur, Jabalpur...",4.697502e+08,SO-22230601-23556,NaN,NaN,GMDEV454704_HOVRCEGW6S,999999999,back,"Autogold Fog Lamp, FOG LAMP ASSY VERNA FLUDIC ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146553,146553,GMDJL590867,DVQOF1ERMF,"Bosch Brake Shoe, BRAKE SHOE F002H23698",F002H23698,BOSCH,1816.0,681.0,2.0,454.0,...,retailer,"LIMBDI,GARAGE,FRONT OF HONEST HOTEL , SHOP NO ...",7.015777e+08,SO-22230831-62063,GET20OFF,Coupon Applied Successfully,GMDJL590867_DVQOF1ERMF,999999999,back,"Bosch Brake Shoe, BRAKE SHOE F002H23698"
146554,146554,GMDJL590867,N12FCNE74V,"Talbros Inner Tie Rod, TIE ROD END PAIR LH RH ...",TRE02009NPR,TALBROS,1242.0,1018.0,1.0,224.0,...,retailer,"LIMBDI,GARAGE,FRONT OF HONEST HOTEL , SHOP NO ...",7.015777e+08,SO-22230831-62063,GET20OFF,Coupon Applied Successfully,GMDJL590867_N12FCNE74V,999999999,back,"Talbros Inner Tie Rod, TIE ROD END PAIR LH RH ..."
146555,146555,GMDJL590867,QH526VT9QL,"Bosch Brake Shoe, REAR BRAKE SHOE SET F002H23675",F002H23675,BOSCH,480.0,360.0,1.0,120.0,...,retailer,"LIMBDI,GARAGE,FRONT OF HONEST HOTEL , SHOP NO ...",7.015777e+08,SO-22230831-62063,GET20OFF,Coupon Applied Successfully,GMDJL590867_QH526VT9QL,999999999,forward,"Bosch Brake Shoe, REAR BRAKE SHOE SET F002H23675"
146556,146556,GMDJL590867,RQFQ7NGNSF,"Bosch Front Brake Pad, FRONT BRAKE PAD SET DIS...",F002H23684,BOSCH,1700.0,1241.0,1.0,459.0,...,retailer,"LIMBDI,GARAGE,FRONT OF HONEST HOTEL , SHOP NO ...",7.015777e+08,SO-22230831-62063,GET20OFF,Coupon Applied Successfully,GMDJL590867_RQFQ7NGNSF,999999999,forward,"Bosch Front Brake Pad, FRONT BRAKE PAD SET DIS..."


In [31]:
order=data[['stockone_order_id','sku_code','quantity','product_title']]
order=order.groupby(['stockone_order_id'],as_index=False).agg({'sku_code':'count','quantity':'sum','product_title':', '.join})

order.shape



(66149, 4)

In [33]:
order=order[order['sku_code']<=10]
order=order[order['sku_code']>1]
order.shape

(17267, 4)

In [34]:
order

,stockone_order_id,sku_code,quantity,product_title
4,GMDAA459868,2,4.0,"DISK COMP. FRICTION, DISK COMP. PRESSURE"
8,GMDAA466830,3,3.0,"MGP Headlight Bulb, BULB (12V,21W) 09471M12045..."
9,GMDAA469436,2,3.0,"GMC (Chevrolet, Opel) Handbrake Cable, CABLE P..."
11,GMDAA470378,2,5.0,"BRACKET-FR BUMPER UPR SIDE MTG, MOBIS (Hyundai..."
13,GMDAA473418,2,5.0,"Tata Door Switch, DOOR SWITCH WITH PIGTAIL 287..."
...,...,...,...,...
66129,GMDZZ568312,2,2.0,"ASSY AIR FILTER COMPLETE - NANO - SUBROS, ASSY..."
66135,GMDZZ575351,4,4.0,"MGP Washer Fluid Pipe, INLET SET,WASHER WITH C..."
66141,GMDZZ578388,2,2.0,"MGP Engine Under Cover, COVER,ENGINE UNDER SID..."
66144,GMDZZ584877,2,2.0,"GoMechanic Safetec Front Brake Pad, Eurorepar ..."


In [35]:
order.columns

Index(['stockone_order_id', 'sku_code', 'quantity', 'product_title'], dtype='object')

In [36]:
order=order[order['product_title'].str.contains('WIPER')==False ]

In [37]:
order.shape

(15539, 4)

In [61]:
order.head()

,stockone_order_id,sku_code,quantity,product_title
4,GMDAA459868,2,4.0,"DISK COMP. FRICTION, DISK COMP. PRESSURE"
8,GMDAA466830,3,3.0,"MGP Headlight Bulb, BULB (12V,21W) 09471M12045..."
9,GMDAA469436,2,3.0,"GMC (Chevrolet, Opel) Handbrake Cable, CABLE P..."
11,GMDAA470378,2,5.0,"BRACKET-FR BUMPER UPR SIDE MTG, MOBIS (Hyundai..."
13,GMDAA473418,2,5.0,"Tata Door Switch, DOOR SWITCH WITH PIGTAIL 287..."


In [62]:
order['stockone_order_id'].nunique()

15539

In [38]:
order.to_csv("orders.csv")

In [39]:
kam_ke_order=list(order['stockone_order_id'].unique())

In [40]:
data=data[data['stockone_order_id'].isin(kam_ke_order)]
#data1=data[['title','quantity']]


In [41]:
len(data['stockone_order_id'].unique())

15539

In [42]:
#data = data.dropna()
data.shape

(48611, 44)

In [44]:
len(data['stockone_order_id'].unique())

15539

In [71]:
pivot = pd.pivot_table(data=data, 
                        index=["stockone_order_id"], 
                        columns=["title"], 
                        values= "quantity",
                        aggfunc='sum')


pivot = pivot.fillna(0)

pivot

title,"ECTOR SHIFTER SHAFT ,4X2 (FLAT END)",WHEEL ARCH SPATS;FRONT BUMPER;RH,WHEEL SPATS;REAR BUMPER;RH,"""MASCOT """"LX""""""",".CAP, RADIATOR",".COVER,MIRROR VISOR,L",".COVER,MIRROR VISOR,R",.ELEMENT,".GASKET, THROTTLE BODY",".KNOB, A/C SWITCH",...,"Zip Oil Filter, OIL FILTER ZO1351","Zip Oil Filter, OIL FILTER ZO1401","Zip Oil Filter, OIL FILTER ZO1402","Zip Oil Filter, OIL FILTER ZO1403","Zip Oil Filter, OIL FILTER ZO1431","Zip Oil Filter, OIL FILTER ZO1461","Zip Oil Filter, OIL FILTER ZO1462","Zip Oil Filter, OIL FILTER ZO1532","Zip Oil Filter, OIL FILTER ZO1533","Zip Oil Filter, OIL FILTER ZO1534"
stockone_order_id,,,,,,,,,,,,,,,,,,,,,
GMDAA459868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GMDAA466830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GMDAA469436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GMDAA470378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GMDAA473418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GMDZZ568312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GMDZZ575351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GMDZZ578388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# pivot = pivot.reset_index()
# pivot

In [73]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

pivot1 = pivot.applymap(encode_units)


In [ ]:
pivot1

In [78]:
pivot1_filter_plus = pivot1[(pivot1 > 0).sum(axis=1) > 2]
pivot1_filter_plus

title,"ECTOR SHIFTER SHAFT ,4X2 (FLAT END)",WHEEL ARCH SPATS;FRONT BUMPER;RH,WHEEL SPATS;REAR BUMPER;RH,"""MASCOT """"LX""""""",".CAP, RADIATOR",".COVER,MIRROR VISOR,L",".COVER,MIRROR VISOR,R",.ELEMENT,".GASKET, THROTTLE BODY",".KNOB, A/C SWITCH",...,"Zip Oil Filter, OIL FILTER ZO1351","Zip Oil Filter, OIL FILTER ZO1401","Zip Oil Filter, OIL FILTER ZO1402","Zip Oil Filter, OIL FILTER ZO1403","Zip Oil Filter, OIL FILTER ZO1431","Zip Oil Filter, OIL FILTER ZO1461","Zip Oil Filter, OIL FILTER ZO1462","Zip Oil Filter, OIL FILTER ZO1532","Zip Oil Filter, OIL FILTER ZO1533","Zip Oil Filter, OIL FILTER ZO1534"
stockone_order_id,,,,,,,,,,,,,,,,,,,,,
GMDAA466830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMDAA488680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMDAA495563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMDAA496347,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMDAA497361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GMDZZ529521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMDZZ535139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMDZZ539933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
0.01/5

0.002

In [35]:
frequent_itemsets_plus = apriori (pivot1_filter_plus, min_support = 0.002,use_colnames =True).sort_values('support', ascending = False).reset_index(drop=True)

frequent_itemsets_plus["length"] = frequent_itemsets_plus["itemsets"].apply(lambda x: len(x))

frequent_itemsets_plus

,support,itemsets,length
0,0.034658,(GoMechanic WINDSHIELD WASHER (50 ML) Windshie...,1
1,0.031313,(GoMechanic SMARTCOOL GREEN COOLANT (1L) Coola...,1
2,0.023016,"(MOBIS (Hyundai, Kia) Oil Filter, FILTER ASSY-...",1
3,0.016325,"(MGP Oil Filter, FILTER ASSY,OIL 16510M65L10)",1
4,0.014184,"(Toyota Oil Filter, MVP ELEMENT KIT OIL 04152Y...",1
...,...,...,...
428,0.002007,"(MOBIS (Hyundai, Kia) Front Brake Pad, PAD KIT...",1
429,0.002007,"(MOBIS (Hyundai, Kia) Bumper Bracket, BRACKET-...",1
430,0.002007,"(Tata Arm Bush, INDIGO FRONT SUS BUSH KIT G277...",1
431,0.002007,"(MOBIS (Hyundai, Kia) Stabilizer Bush, BUSH-ST...",1


In [36]:
from mlxtend.frequent_patterns import association_rules 

In [37]:
freq=frequent_itemsets_plus[frequent_itemsets_plus['length']>1]
freq.to_csv("freq.csv")

In [38]:
freq_2=freq[freq['length']==2]
freq_3=freq[freq['length']==3]
freq_4=freq[freq['length']==4]
freq_5=freq[freq['length']==5]

In [39]:
freq_2=freq_2.reset_index()
freq_2.drop(["index"],axis=1,inplace=True)
for i in range(len(freq_2)):
    freq_2['itemsets'][i]=list(freq_2['itemsets'][i])
freq_2[['first_sku','second_sku']] = freq_2.itemsets.apply(pd.Series)
freq_2=freq_2[['first_sku','second_sku','support']]
freq_2['support']=freq_2['support']*100
freq_2

,first_sku,second_sku,support
0,GoMechanic SMARTCOOL GREEN COOLANT (1L) Coolan...,GoMechanic WINDSHIELD WASHER (50 ML) Windshiel...,0.695838
1,"Toyota Oil Filter, MVP ELEMENT KIT OIL 04152YZZA8","Toyota Air Filter, ELEMENT SUB ASSY AI 178010N050",0.628931
2,"Toyota Clutch Pressure Plate, COVER ASSY CLUTC...","Toyota Clutch Friction Disc, DISC ASSY CLUTCH ...",0.562023
3,"Honda Air Filter, ELEMENT ASSY AIR CLEANER 172...","Honda Oil Filter, CARTRIDGE OIL FILTER PT DUTA...",0.562023
4,"Toyota Engine Oil, MOTOR OIL 5L L0888083118","Toyota Engine Oil, MOTOR OIL 1L L0888083119",0.548642
...,...,...,...
61,"MOBIS (Hyundai, Kia) Air Filter, FILTER-AIR CL...","MOBIS (Hyundai, Kia) Air Filter, FILTER-AIR CL...",0.200723
62,"MOBIS (Hyundai, Kia) Air Filter, FILTER-AIR CL...","MOBIS (Hyundai, Kia) Air Filter, FILTER-AIR CL...",0.200723
63,"MOBIS (Hyundai, Kia) Air Filter, FILTER-AIR CL...","MOBIS (Hyundai, Kia) Oil Filter, FILTER ASSY-E...",0.200723
64,"MOBIS (Hyundai, Kia) Air Filter, FILTER-AIR CL...","MOBIS (Hyundai, Kia) Oil Filter, SERVICE KIT-O...",0.200723


In [40]:
freq_2.to_csv("freq_2.csv")

In [41]:
freq_3=freq_3.reset_index()
freq_3.drop(["index"],axis=1,inplace=True)
for i in range(len(freq_3)):
    freq_3['itemsets'][i]=list(freq_3['itemsets'][i])
    
freq_3[['first_sku','second_sku','third_sku']] = freq_3.itemsets.apply(pd.Series)
freq_3=freq_3[['first_sku','second_sku','third_sku','support']]
freq_3['support']=freq_3['support']*100
freq_3

,first_sku,second_sku,third_sku,support
0,"Toyota Oil Filter, OIL FILTER 75 85 90915YZZD2","Toyota Air Filter, ELEMENT AIR CLEANER 178010C030","Toyota Fuel Filter, FUEL FILTER 233900L041",0.200723
1,"Toyota Oil Filter, MVP ELEMENT KIT OIL 04152YZZA8","Toyota Air Filter, ELEMENT SUB ASSY AI 178010N050","Toyota Fuel Filter, ELEMENT SUB ASSY FUE 23390...",0.200723


In [42]:
freq_3.to_csv("freq_3.csv")

In [43]:
freq_4=freq_4.reset_index()
freq_4.drop(["index"],axis=1,inplace=True)
for i in range(len(freq_4)):
    freq_4['itemsets'][i]=list(freq_4['itemsets'][i])
    
freq_4[['first_sku','second_sku','third_sku','forth_sku']] = freq_4.itemsets.apply(pd.Series)
freq_4=freq_4[['first_sku','second_sku','third_sku','forth_sku','support']]
freq_4['support']=freq_4['support']*100
freq_4

ValueError: Columns must be same length as key

In [ ]:
freq_4.to_csv("freq_4.csv")

In [ ]:
freq_5=freq_5.reset_index()
freq_5.drop(["index"],axis=1,inplace=True)
for i in range(len(freq_5)):
    freq_5['itemsets'][i]=list(freq_5['itemsets'][i])
    freq_5
freq_5[['first_sku','second_sku','third_sku','forth_sku','fifth_sku']] = freq_5.itemsets.apply(pd.Series)
freq_5=freq_5[['first_sku','second_sku','third_sku','forth_sku','fifth_sku','support']]
freq_5['support']=freq_5['support']*100
freq_5

In [ ]:
freq_5.to_csv("freq_5.csv")

In [ ]:
data[data['product_title'].isin(['GoMechanic WINDSHIELD WASHER (50 ML) Windshield Washer Fluid GMUNZZWS001','GoMechanic SMARTCOOL GREEN COOLANT (1L) Coolant GMUNZZCL001'])]['price'].sum()


In [ ]:
data[data['product_title'].isin(['GoMechanic WINDSHIELD WASHER (50 ML) Windshield Washer Fluid GMUNZZWS001','GoMechanic SMARTCOOL GREEN COOLANT (1L) Coolant GMUNZZCL001'])]['stockone_order_id'].count()


In [49]:

rules=association_rules(frequent_itemsets_plus, metric = "lift",min_threshold=0.0002).sort_values("lift", ascending = False).reset_index(drop =True)

In [79]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))

In [80]:
#rules1=rules[rules['antecedent_len']>2]
#rules1=rules1[rules1['consequents_len']>2]     
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequents_len
0,"(MOBIS (Hyundai, Kia) Bumper Bracket, BRACKET-...","(MOBIS (Hyundai, Kia) Bumper Bracket, BRACKET-...",0.002007,0.002409,0.002007,1.000000,415.166667,0.002002,inf,1,1
1,"(MOBIS (Hyundai, Kia) Bumper Bracket, BRACKET-...","(MOBIS (Hyundai, Kia) Bumper Bracket, BRACKET-...",0.002409,0.002007,0.002007,0.833333,415.166667,0.002002,5.987957,1,1
2,"(Honda Control Arm, ARM ASSY L FR LOWER 51360T...","(Honda Control Arm, ARM ASSY R FR LOWER 51350T...",0.002409,0.002542,0.002275,0.944444,371.464912,0.002269,17.954235,1,1
3,"(Honda Control Arm, ARM ASSY R FR LOWER 51350T...","(Honda Control Arm, ARM ASSY L FR LOWER 51360T...",0.002542,0.002409,0.002275,0.894737,371.464912,0.002269,9.477118,1,1
4,"(MGP Bumper Bracket, HOLDER, FR BUMPER SIDE LH...","(MGP Bumper Bracket, HOLDER, FR BUMPER SIDE RH...",0.002810,0.002542,0.002409,0.857143,337.127820,0.002402,6.982203,1,1
...,...,...,...,...,...,...,...,...,...,...,...
139,(GoMechanic BRAKE OIL DOT-4 250 ML Brake Fluid...,(GoMechanic WINDSHIELD WASHER (50 ML) Windshie...,0.010438,0.034658,0.002676,0.256410,7.398277,0.002315,1.298218,1,1
140,(GoMechanic WINDSHIELD WASHER (50 ML) Windshie...,(GoMechanic SMARTCOOL GREEN COOLANT (1L) Coola...,0.034658,0.031313,0.006958,0.200772,6.411840,0.005873,1.212029,1,1
141,(GoMechanic SMARTCOOL GREEN COOLANT (1L) Coola...,(GoMechanic WINDSHIELD WASHER (50 ML) Windshie...,0.031313,0.034658,0.006958,0.222222,6.411840,0.005873,1.241154,1,1
142,"(MOBIS (Hyundai, Kia) Oil Filter, FILTER ASSY-...","(MGP Oil Filter, FILTER ASSY,OIL 16510M65L10)",0.023016,0.016325,0.002141,0.093023,5.698056,0.001765,1.084564,1,1


In [81]:
rules.to_csv("association_rules.csv")